In [1]:
!pip install --upgrade pip
!pip install sentence-transformers
!pip install requests
!pip install opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 21.7 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 91.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 64.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 50.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 57.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 70.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 

In [2]:
import torch
print("CUDA disponible :", torch.cuda.is_available())
print("Nom du GPU :", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "Aucun GPU détecté")


CUDA disponible : True
Nom du GPU : Tesla T4


In [12]:
import requests
from sentence_transformers import SentenceTransformer, util
from PIL import Image
import glob
import os
import cv2
import timeit
import torch


def has_transparency(img_path):
    img = Image.open(img_path)
    if img.info.get("transparency", None) is not None:
        return True
    if img.mode == "P":
        transparent = img.info.get("transparency", -1)
        for _, index in img.getcolors():
            if index == transparent:
                return True
    elif img.mode == "RGBA":
        extrema = img.getextrema()
        if extrema[3][0] < 255:
            return True

    return False

# Function to check if the image has an accepted aspect ratio
def has_accepted_aspect_ratio(image_path):
    # Define accepted aspect ratios and a tolerance for floating-point precision
    accepted_aspect_ratios = [(16, 9), (4, 3)]
    tolerance = 0.01  # Define a small tolerance for floating-point comparisons

    # Load the image
    image = cv2.imread(image_path)

    if image is None:
        print(f"Error: Unable to open image at {image_path}")
        return False

    # Get the dimensions of the image
    height, width = image.shape[:2]

    # Calculate the aspect ratio
    aspect_ratio = width / height

    # Check if the aspect ratio matches any of the accepted ratios within the tolerance
    for w, h in accepted_aspect_ratios:
        target_ratio = w / h
        if abs(aspect_ratio - target_ratio) < tolerance:
            return True  # The image matches one of the accepted aspect ratios

    return False  # Reject if no match found

def filter_image():
    garbage =[]
    image_names = list(glob.glob('./*.jpg'))
    print("Number of images unfiltered:", len(image_names))
    encoded_image = model.encode([Image.open(filepath) for filepath in image_names], batch_size=1, convert_to_tensor=True, show_progress_bar=True,device=device)

    processed_images = util.paraphrase_mining_embeddings(encoded_image)
    NUM_SIMILAR_IMAGES = len(image_names)

    print('Finding duplicate images...')
    duplicates = [image for image in processed_images if image[0] >= filtering_treshold]

    # On met a la poubelle les image trop similaire
    for score, image_id1, image_id2 in duplicates[0:NUM_SIMILAR_IMAGES]:
        if image_names[image_id1] not in garbage:
            garbage.append(image_names[image_id1])
        print("\nScore: {:.3f}%".format(score * 100))
        print(image_names[image_id1],flush=True)
        print(image_names[image_id2],flush=True)

    #on met a la poubelle les image transparentes ou avec un ratio non voulu
    for k in image_names :
        if has_transparency(k) and k not in garbage:
            garbage.append(k)
        elif k not in garbage and has_accepted_aspect_ratio(k)==False:
            garbage.append(k)


    #On supprime toute les image non necessaire
    for k in garbage :
        try:
            os.remove(k)
            print(f"{k} has been deleted successfully.")
            image_names.remove(k)
        except FileNotFoundError:
            print(f"{k} does not exist.")
        except PermissionError:
            print(f"Permission denied to delete {k}.")
        except Exception as e:
            print(f"Error occurred while deleting file: {e}")
    print("Number of images filtered:", len(image_names))

def encode_les_images():
    # Get the list of images in the directory
    image_names = list(glob.glob('./*.jpg'))
    # Encode all other images in the directory, and move them to DirectML device
    encoded_images = model.encode([Image.open(filepath) for filepath in image_names],
                                  batch_size=1,
                                  convert_to_tensor=True,
                                  show_progress_bar=True,
                                  device=device)
    return encoded_images

#Cette fonction calcule la similarité entre toute les pairs le nombre de comparaison est donc exponentielle ~1sec pour cheque 1 frame avec 7 image
#!Batch : 1.27 it/s avec 5 image en moyenne cpu batch size = 4
#!Batch : 5.4 it/s avec 5 image en moyenne cpu batch size = 2
#!Batch : 9 it/s {entre 7 et 10} avec 5 image en moyenne gpu batch size = 2 | Batches 3/3
#!Batch : 15 it/s {entre 11 et 14.5} avec 5 image en moyenne gpu batch size = 2 | Batches 5/5

def find_matching_imagesV1(frame_path):
    image_names = list(glob.glob('./*.jpg'))

    # Next we compute the embeddings
    # To encode an image, you can use the following code:
    encoded_image = model.encode([Image.open(filepath) for filepath in image_names], batch_size=1, convert_to_tensor=True, show_progress_bar=True,device=device)
    # Now we run the clustering algorithm. This function compares images aganist
    # all other images and returns a list with the pairs that have the highest
    # cosine similarity score
    processed_images = util.paraphrase_mining_embeddings(encoded_image)
    NUM_SIMILAR_IMAGES = len(image_names)
    # =================
    # NEAR DUPLICATES
    # =================
    print('Finding near duplicate images...')

    for score, image_id1, image_id2 in processed_images[0:NUM_SIMILAR_IMAGES]:
        if score*100 < matchin_images_treshold:
            return 0
        print(image_names[image_id1])
        print(image_names[image_id2])
        if image_names[image_id1]==frame_path or image_names[image_id2]==frame_path:
            return (score * 100)
    return 0

#!Batch : 3 it/s avec 5 image en moyenne avec batch size =4 batches : 1/1 avec gpu
#!Batch : 6 it/s {entre 5 et 7 it/s} avec 5 image en moyenne avec batch size =2 batches 2/2 avec gpu
#!Batch : 12 it/s avec 5 image en moyenne avec batch size =1 batches 4/4 avec gpu
def find_matching_imagesV2(frame_path,encoded_images):
    # Encode the image you want to compare, and move it to DirectML device
    frame_image_embedding = model.encode([Image.open(frame_path)], batch_size=16 ,convert_to_tensor=True, device=device)

    # Compute cosine similarities between the frame image and the other images (on AMD GPU)
    similarities = util.cos_sim(frame_image_embedding, encoded_images)

    # Go through the similarity scores
    max=0
    for i, score in enumerate(similarities[0]):
        score_percentage = score.item() * 100  # Convert the similarity score to percentage
        if score_percentage > max:
            max = score_percentage

    # Return 0 if no similar image meets the threshold
    return max



def get_images(query,api_key,cx,number_of_request):
    for k in range(number_of_request):
        # The API endpoint
        url = f"https://www.googleapis.com/customsearch/v1?q={query}&key={api_key}&cx={cx}&searchType=image&imgType=photo&hl=en&num=10&start={(k*10)+1}"

        # Making the HTTP GET request
        response = requests.get(url)

        # Parsing the JSON response
        results = response.json()

        filename="downloaded_image"
        chiffre=k*10

        # Print the image results
        for item in results.get("items", []):
            image_url=item['link']
            response = requests.get(image_url)
            real_filename=filename+str(chiffre)+".jpg"
            chiffre=chiffre+1
            # Check if the request was successful
            if response.status_code == 200:
                # Open a file in binary write mode and write the content
                with open(real_filename, "wb") as file:
                    file.write(response.content)
                    print("Image downloaded successfully!")
            else:
                print("Failed to download the image.")

def new_frame_gap(prob,prev_prob,nb_frame,current_gap):
    for k in seuil.keys():
            if prob<k:
                gap=seuil[k]
                new_frame=nb_frame
                nb_etape=0
                if file_de_voiture:
                    if prob!=0 and prob>prev_prob and prev_prob < k-5:
                        while(new_frame-(gap*60)>nb_frame-(current_gap*60)):
                            new_frame=new_frame-(gap*60)
                            nb_etape+=1
                        print("Deceleration, on recheck a partir de la frame " +str(new_frame) +" alors qu'on était a la frame " + str(nb_frame) + " On est revenu en arrière de :" + str(nb_etape) + "\n")
                break
    return gap,new_frame

def scan_video(videopath):
    # Open the video file
    video = cv2.VideoCapture(videopath)
    # Initialize variables
    found_timestamps = {}
    video_lenght = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    encode_images = encode_les_images()
    current_frame=0
    prev_prob=0
    frame_gap=300 #Can be changed

    # Loop through each frame of the video
    while (current_frame < video_lenght):

        video.set(cv2.CAP_PROP_POS_FRAMES, current_frame)
        ret, frame = video.read()

        if not ret:
            break
        print('minute elapsed', (current_frame / 3600))

        #au cas ou il y a unne erreur au moment de la réécriture on relance le processus
        if cv2.imwrite('extracted_frame.jpg', frame) == False:
            cv2.imwrite('extracted_frame.jpg', frame)

        # Perform template matching
        prob = find_matching_imagesV2('./extracted_frame.jpg',encode_images)
        print(prob)

        # If match is above threshold, record the timestamp
        if prob >= matchin_images_treshold:
            timestamp = video.get(cv2.CAP_PROP_POS_MSEC) / 1000.0  # in seconds
            found_timestamps[round(timestamp/60,2)]=prob

        frame_gap,current_frame=new_frame_gap(prob,prev_prob,current_frame,frame_gap)
        prev_prob = prob
        current_frame+=frame_gap*60

    video.release()
    os.remove("extracted_frame.jpg")

    # Output the found timestamps
    print("image found at timestamps:", found_timestamps)


#? KEY PARAMETERS

VIDEO_SCANNED = "/content/drive/MyDrive/Ratchet_et_clank_2_part_4.mp4"
# Your API key
api_key = ""
# Your Custom Search Engine ID
cx = ""
# The search query
query = ""
# Number of request (10 images per request)
num_requests = 2
# seuil d'acceptation pour la similarité entre image valeur comprise entre 0 et 100
# 85 pour le modele complet semble bien et 88 pour le modele half semble similaire au modèle complet
matchin_images_treshold = 88
# les images ayant plus de X% de ressemblance seront supprimé valeur comprise entre 0 et 1
filtering_treshold = 0.80
#Seuil pour la vitesse adaptative [seuil en pourcentage : vitesse en seconde]
seuil={75:20,80:10,85:5,90:2,100:2}
#Décide si on utilise la méthode file de voiture expliquer dans performane.txt| Permet une approche intélligente qui mixe rapidité et fiabilité.
file_de_voiture=True



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
# Charger le modèle et l’envoyer sur le GPU NVIDIA
model = SentenceTransformer('clip-ViT-B-32').to(device)
#get_images(query,api_key,cx,num_requests)
#filter_image()
total_time = timeit.timeit(lambda : scan_video(VIDEO_SCANNED), number=1)
print(f"Mean time {total_time:.3e} s")


Using device: cuda


Batches:   0%|          | 0/7 [00:00<?, ?it/s]

minute elapsed 0.0
59.084731340408325
Deceleration, on recheck a partir de la frame -16800 alors qu'on était a la frame 0 On est revenu en arrière de :14

minute elapsed -4.333333333333333
57.155489921569824
minute elapsed -4.0
56.82951211929321
minute elapsed -3.6666666666666665
57.550132274627686
Deceleration, on recheck a partir de la frame -13200 alors qu'on était a la frame -13200 On est revenu en arrière de :0

minute elapsed -3.3333333333333335
57.47154355049133
minute elapsed -3.0
57.3749840259552
minute elapsed -2.6666666666666665
57.99402594566345
Deceleration, on recheck a partir de la frame -9600 alors qu'on était a la frame -9600 On est revenu en arrière de :0

minute elapsed -2.3333333333333335
58.17163586616516
Deceleration, on recheck a partir de la frame -8400 alors qu'on était a la frame -8400 On est revenu en arrière de :0

minute elapsed -2.0
58.43225717544556
Deceleration, on recheck a partir de la frame -7200 alors qu'on était a la frame -7200 On est revenu en arr